In [1]:
import numpy as np
import scipy.sparse as sp
import random
from tqdm import tqdm

In [2]:
fio = open("train.txt", 'r')
rows = []
cols = []
data = []
for l in fio.readlines():
    args = l.split('\t')
    args = [x for x in args if len(x) > 0]
    i = int(args[0])-1
    j = int(args[1])-1
    p = float(args[2])
    rows.append(i)
    cols.append(j)
    data.append(p)
fio.close()
R_train = sp.csr_matrix((data, (rows, cols))).toarray()


In [3]:

class IALS:
   
    @staticmethod
    def __log_confidence(R,  alpha, eps):
        return (np.ones(R.shape) + alpha * np.log(np.ones(R.shape) + R_train / eps))
    
    def __bias_normalisation(self,  R):
        gr_mask = R > 0
        eq_mask = R == 0
        mean = R[gr_mask].mean()
        R_unbiased = R * gr_mask - gr_mask * mean
        user_bias = (R_unbiased.sum(1) / gr_mask.sum(1)).reshape(-1, 1)
        R_unuserbiased = R_unbiased * gr_mask - gr_mask * user_bias
        gr_mask_sum_0 = gr_mask.sum(0) + 0.0000001
        item_bias = (R_unuserbiased.sum(0) / gr_mask_sum_0).reshape(1, -1)
        
        P = R + eq_mask * (user_bias + item_bias + mean) 


        return P



    def __init__(self, max_epoch=100, embedding_size=6, alpha=0.1, l2reg=1,  eps=0.1,
                random_state=42):

        self.max_epoch = max_epoch
        self.embedding_size = embedding_size
        self.alpha = alpha
        self.l2reg = l2reg
        self.random_state = random_state
        self.eps = eps
       
        np.random.seed(self.random_state)


    def fit(self, R_train):
      

        C = IALS.__log_confidence(R_train,  self.alpha, self.eps)

        P = self.__bias_normalisation(R_train)
        Cm1 = C - 1

        lambdaI = np.eye(self.embedding_size + 1, self.embedding_size + 1) * self.l2reg  
        X = np.hstack([np.ones((R_train.shape[0], 1)), np.random.random_sample((R_train.shape[0], self.embedding_size))])

        Y = np.hstack([np.ones((R_train.shape[1], 1)), np.random.random_sample((R_train.shape[1], self.embedding_size))])

        beta = np.zeros((P.shape[0], 1)).reshape(-1,1) 
        gamma = np.zeros((P.shape[1], 1)).reshape(1,-1)  


        n_user = (R_train > 0).sum(1)
        n_item = (R_train > 0).sum(0)

        for epoch in range(self.max_epoch):

            YtY = np.matmul(Y.T, Y)
            d = C *  (P - gamma)
            X[:,0] = beta.squeeze()

            for i in range(X.shape[0]):  
                X[i, :] = np.matmul( np.matmul(np.linalg.inv(YtY + lambdaI * n_user[i] + np.matmul(Y.T * Cm1[i, :], Y)),\
                                               Y.T), d[i, :].reshape(-1, 1)).ravel()
            beta = X[:, 0].copy().reshape(-1,1)
            X[:, 0] = 1
            XtX = np.matmul(X.T, X)
            d = C * (P - beta)
            Y[:,0] = gamma.squeeze()

            for j in range(Y.shape[0]):  
                Y[j, :] = np.matmul(np.matmul(np.linalg.inv(XtX + lambdaI * n_item[j]  + np.matmul(X.T * Cm1[:, j], X)),\
                                              X.T), d[:, j].reshape(-1, 1)).ravel()
            gamma = Y[:, 0].copy().reshape(1, -1)
            Y[:, 0] = 1

        result = np.matmul(X[:, 1:], Y[:, 1:].T) + beta + gamma 
        result = result.copy()
        result[result > 5] = 5
        result[result < 1] = 1
        self.result = result

    
    def predict(self):
        return self.result

In [4]:
def train_test_split(R, test_ratio=0.1):
 
    data_size = R.shape[0] * R.shape[1]
    nonzero_items = np.arange(data_size)[R.ravel() > 0]
    nonzero_size = nonzero_items.shape[0]
    test_size = int(test_ratio * nonzero_size)
    np.random.shuffle(nonzero_items)
    train_items = nonzero_items[test_size:]
    test_items = nonzero_items[:test_size]
    train_indices = np.unravel_index(train_items, R.shape)
    test_indices = np.unravel_index(test_items, R.shape)
    train_mask = np.zeros(R.shape)
    for i in range(train_indices[0].shape[0]):
        train_mask[train_indices[0][i], train_indices[1][i]] = True
    test_mask = np.zeros(R.shape)
    for i in range(test_indices[0].shape[0]):
        test_mask[test_indices[0][i], test_indices[1][i]] = True
    return R * train_mask, R * test_mask

In [5]:
X_tr, X_val = train_test_split(R_train)

In [6]:
best_mse = np.inf
for alpha in np.arange(10, 50, 5):
    for l2reg in np.arange(1,12,1):
           for eps in np.arange(0.01, 0.15, 0.01):
                ials1 = IALS(max_epoch=50,
                embedding_size=4,
                alpha=alpha,
                l2reg=l2reg,
                eps=eps) 
                res = ials1.fit(X_tr)
                cur_mse = mean_squared_error(res[np.where(X_val != 0)], X_val[np.where(X_val != 0)])
                if cur_mse < best_mse:                  
                    best_alpha = alpha
                    best_l2reg = l2reg
                    best_eps = eps
                    best_mse = cur_mse

In [7]:

print(best_alpha)
print(best_l2reg)
print(best_eps)

35
10
0.05


In [8]:
best_mse = np.inf
for alpha in np.arange(10, 50, 5):
    for l2reg in np.arange(1,12,1):
           for eps in np.arange(0.01, 0.15, 0.01):
                ials1 = IALS(max_epoch=35,
                embedding_size=15,
                alpha=alpha,
                l2reg=l2reg,
                eps=eps) 
                res = ials1.fit(X_tr)
                cur_mse = mean_squared_error(res[np.where(X_val != 0)], X_val[np.where(X_val != 0)])
                if cur_mse < best_mse:
            
                    best_alpha = alpha
                    best_l2reg = l2reg
                    best_eps = eps
                    best_mse = cur_mse

In [9]:
print(best_alpha)
print(best_l2reg)
print(best_eps)

35
20
0.1


In [10]:
best_mse = np.inf
for alpha in np.arange(10, 50, 5):
    for l2reg in np.arange(1,12,1):
           for eps in np.arange(0.01, 0.15, 0.01):
                ials1 = IALS(max_epoch=35,
                embedding_size=6,
                alpha=alpha,
                l2reg=l2reg,
                eps=eps) 
                res = ials1.fit(X_tr)
                cur_mse = mean_squared_error(res[np.where(X_val != 0)], X_val[np.where(X_val != 0)])
                if cur_mse < best_mse:
                    best_alpha = alpha
                    best_l2reg = l2reg
                    best_eps = eps
                    best_mse = cur_mse

In [11]:
print(best_alpha)
print(best_l2reg)
print(best_eps)

35
10
0.1


In [12]:
first = IALS(max_epoch=35,
            embedding_size=6,
            alpha=35,
            l2reg=10,
            eps=0.1)

second = IALS(max_epoch=35,
            embedding_size=15,
            alpha=35,
            l2reg=10,
            eps=0.1)

third = IALS(max_epoch=50,
            embedding_size=4,
            alpha=35,
            l2reg=10,
            eps=0.05)


first.fit(R_train)
second.fit(R_train)
third.fit(R_train)

first_res = first.predict()
second_res = second.predict()
third_res = third.predict()

res = (first_res + second_res + third_res ) / 3
submission_name = ("submission_{}.txt".format(7))
with open("test.txt", 'r') as file_in:
    with open(submission_name, 'w') as file_out:
        file_out.write("Id,Score\n")
        for id, line in enumerate(file_in):
            ui = [int(a) for a in line.strip().split('\t')]
            file_out.write("{},{}\n".format(id + 1, res[ui[0] - 1, ui[1] - 1]))